In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
import time

print("--- Novel MST-Based Purity Score Algorithm ---")
print("This script will add a new 'purity_score' feature to the fetal health dataset.")

# -----------------------------------------------------------------
# MODULE 1: DISJOINT SET UNION (DSU) STRUCTURE
# This is required to detect cycles in Kruskal's algorithm
# -----------------------------------------------------------------
class DSU:
    """
    A class for the Disjoint Set Union (DSU) data structure.
    Also known as Union-Find.
    """
    def __init__(self, n):
        # Initialize parent array, each node is its own parent
        self.parent = list(range(n))

    def find(self, i):
        """Finds the root representative of node i's set (with path compression)"""
        if self.parent[i] == i:
            return i
        # Path compression: set parent directly to the root
        self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        """Merges the sets containing nodes i and j"""
        root_i = self.find(i)
        root_j = self.find(j)
        
        if root_i != root_j:
            # Not in the same set, merge them
            self.parent[root_i] = root_j
            return True  # Successful union
        
        return False # Already in the same set (cycle detected)

# -----------------------------------------------------------------
# MODULE 2: DATA LOADING AND PREPARATION
# -----------------------------------------------------------------
try:
    print("\nLoading fetal_health.csv...")
    df = pd.read_csv('fetal_health.csv')
except FileNotFoundError:
    print("Error: 'fetal_health.csv' not found.")
    print("Please place this script in the same folder as your dataset.")
    exit()

# Separate features (X) and labels (y)
# We assume the last column is the target
features_df = df.iloc[:, :-1]
labels = df.iloc[:, -1].values
n = len(df) # Number of records (nodes)
d = features_df.shape[1] # Number of features (dimensions)

print(f"Loaded {n} records with {d} features.")

# CRITICAL STEP: Standardize features
# Distances are only meaningful if features are on the same scale.
print("Standardizing features...")
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_df)

# Epsilon for stable division
epsilon = 1e-6 

# -----------------------------------------------------------------
# MODULE 3: MAIN ALGORITHM SCRIPT
# -----------------------------------------------------------------

# --- 3A: Calculate all edge weights and sort them ---
print(f"Calculating all-pairs distances (total {n*(n-1)//2} edges)...")
start_time = time.time()

all_edges = []
for i in range(n):
    for j in range(i + 1, n):
        # Calculate Euclidean distance using fast numpy operations
        dist = np.sqrt(np.sum((features_scaled[i] - features_scaled[j])**2))
        all_edges.append((dist, i, j)) # (weight, node1, node2)

print(f"  -> Edge calculation took {time.time() - start_time:.2f} seconds.")

# --- 3B: Sort edges by weight ---
print("Sorting all edges...")
start_time = time.time()
all_edges.sort()
print(f"  -> Sorting took {time.time() - start_time:.2f} seconds.")


# --- 3C: Build the MST using Kruskal's Algorithm ---
print("Building Minimum Spanning Tree (MST)...")
start_time = time.time()

dsu = DSU(n)
# This will store the MST structure: {node: [(neighbor, dist), ...]}
mst_adj = {i: [] for i in range(n)} 
mst_edge_count = 0

for dist, u, v in all_edges:
    if dsu.union(u, v):
        # No cycle! Accept this edge.
        mst_adj[u].append((v, dist))
        mst_adj[v].append((u, dist))
        mst_edge_count += 1
        
        # Stop once the MST is complete
        if mst_edge_count == n - 1:
            break

print(f"  -> MST build took {time.time() - start_time:.2f} seconds.")

# --- 3D: Calculate the Purity Score for each node ---
print("Calculating novel Purity Scores...")
purity_scores = []
for i in range(n):
    penalty = 0
    my_label = labels[i]
    
    # Get neighbors *from the MST only*
    for neighbor_id, dist in mst_adj[i]:
        neighbor_label = labels[neighbor_id]
        
        # If neighbor's class is DIFFERENT, add the distance as a penalty
        if neighbor_label != my_label:
            penalty += dist
            
    # Final score is the inverse of the penalty
    score = 1.0 / (penalty + epsilon)
    purity_scores.append(score)

# --- 4: Save Results ---
print("Saving new dataset...")
# Add the new feature to our original dataframe
df['purity_score'] = purity_scores

# Save to a new CSV file
output_filename = 'fetal_health_with_purity.csv'
df.to_csv(output_filename, index=False)

print(f"\n--- SUCCESS! ---")
print(f"A new file '{output_filename}' has been created.")
print("It contains your original data plus the new 'purity_score' feature.")

# Display a sample of the results
print("\nSample of results (Low score = boundary, High score = core):")
print(df[['fetal_health', 'purity_score']].groupby('fetal_health').describe())

--- Novel MST-Based Purity Score Algorithm ---
This script will add a new 'purity_score' feature to the fetal health dataset.

Loading fetal_health.csv...
Loaded 2126 records with 21 features.
Standardizing features...
Calculating all-pairs distances (total 2258875 edges)...
  -> Edge calculation took 8.78 seconds.
Sorting all edges...
  -> Sorting took 2.58 seconds.
Building Minimum Spanning Tree (MST)...
  -> MST build took 1.58 seconds.
Calculating novel Purity Scores...
Saving new dataset...

--- SUCCESS! ---
A new file 'fetal_health_with_purity.csv' has been created.
It contains your original data plus the new 'purity_score' feature.

Sample of results (Low score = boundary, High score = core):
             purity_score                                          \
                    count           mean            std       min   
fetal_health                                                        
1.0                1655.0  918429.063159  273792.537224  0.036882   
2.0            

In [2]:
# === Model training & evaluation: Gaussian Naive Bayes ===
# This cell loads the CSV produced earlier (with `purity_score`),
# runs two experiments (with and without purity_score), prints metrics,
# and saves the model that uses the purity_score.

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

print('\nLoading dataset with purity score...')
df_all = pd.read_csv('fetal_health_with_purity.csv')
print(f"Dataset shape: {df_all.shape}")

# Identify target column (assume same as before)
target_col = 'fetal_health' if 'fetal_health' in df_all.columns else df_all.columns[-1]

# Prepare feature sets
feature_cols = [c for c in df_all.columns if c != target_col]
if 'purity_score' not in feature_cols:
    raise RuntimeError("purity_score column not found in 'fetal_health_with_purity.csv'. Please ensure the previous cell ran and saved the file.")

# Original-features-only (exclude purity_score)
orig_features = [c for c in feature_cols if c != 'purity_score']

X_with = df_all[feature_cols].copy()
X_without = df_all[orig_features].copy()
y = df_all[target_col].values

# Train/test split (stratify if labels are categorical)
Xw_train, Xw_test, yw_train, yw_test = train_test_split(X_with, y, test_size=0.2, random_state=42, stratify=y)
Xo_train, Xo_test, yo_train, yo_test = train_test_split(X_without, y, test_size=0.2, random_state=42, stratify=y)

# Scale features (GaussianNB assumes continuous features; scaling often helps)
scaler_with = StandardScaler()
Xw_train_s = scaler_with.fit_transform(Xw_train)
Xw_test_s = scaler_with.transform(Xw_test)

scaler_without = StandardScaler()
Xo_train_s = scaler_without.fit_transform(Xo_train)
Xo_test_s = scaler_without.transform(Xo_test)

# Helper to train & evaluate
def train_eval(X_train, X_test, y_train, y_test, desc=''):
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, digits=4)
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n=== Results: {desc} ===")
    print(f"Accuracy: {acc:.4f}")
    print("\nClassification report:\n", report)
    print("Confusion matrix:\n", cm)
    return clf, acc

# Run experiments
clf_with, acc_with = train_eval(Xw_train_s, Xw_test_s, yw_train, yw_test, desc='With purity_score')
clf_without, acc_without = train_eval(Xo_train_s, Xo_test_s, yo_train, yo_test, desc='Without purity_score')

# Compare and save the model that performed better (prefer with purity if tie)
if acc_with >= acc_without:
    best_clf = clf_with
    best_scaler = scaler_with
    model_name = 'model/gaussian_nb_with_purity.joblib'
    scaler_name = 'model/scaler_with_purity.joblib'
    chosen = 'with purity_score'
else:
    best_clf = clf_without
    best_scaler = scaler_without
    model_name = 'model/gaussian_nb_without_purity.joblib'
    scaler_name = 'model/scaler_without_purity.joblib'
    chosen = 'without purity_score'

# Ensure model dir exists and save
import os
os.makedirs('model', exist_ok=True)
joblib.dump(best_clf, model_name)
joblib.dump(best_scaler, scaler_name)

print(f"\nSaved best model ({chosen}) to: {model_name}")
print(f"Saved corresponding scaler to: {scaler_name}")

# Quick sample of predictions on test set for sanity
sample_idx = np.random.choice(len(Xw_test_s), min(5, len(Xw_test_s)), replace=False)
print('\nSample predictions (test set):')
print('True\tPredicted')
print([ (int(yw_test[i]), int(clf_with.predict(Xw_test_s)[i])) for i in sample_idx ])



Loading dataset with purity score...
Dataset shape: (2126, 23)

=== Results: With purity_score ===
Accuracy: 0.8192

Classification report:
               precision    recall  f1-score   support

         1.0     0.9826    0.8494    0.9111       332
         2.0     0.4587    0.8475    0.5952        59
         3.0     0.5667    0.4857    0.5231        35

    accuracy                         0.8192       426
   macro avg     0.6693    0.7275    0.6765       426
weighted avg     0.8759    0.8192    0.8355       426

Confusion matrix:
 [[282  43   7]
 [  3  50   6]
 [  2  16  17]]

=== Results: Without purity_score ===
Accuracy: 0.8099

Classification report:
               precision    recall  f1-score   support

         1.0     0.9691    0.8494    0.9053       332
         2.0     0.4476    0.7966    0.5732        59
         3.0     0.5333    0.4571    0.4923        35

    accuracy                         0.8099       426
   macro avg     0.6500    0.7011    0.6569       426
weigh